In [ ]:
# Installation of some requirements
# %pip install albumentations
# %pip install ultralytics

In [ ]:
# Kaggle dataset download
# !kaggle datasets download juanthomaswijaya/indonesian-license-plate-dataset --path data --unzip

In [ ]:
# %matplotlib inline

In [1]:
import numpy as np
import torch # If you use PyTorch later for training
import random
import os

# Choose an integer seed (e.g., 42, 0, or any number)
SEED = 42

def set_all_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    # If you use PyTorch, uncomment these:
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_all_seeds(SEED)

In [ ]:
import cv2
import albumentations as A
from glob import glob

# Paths
INPUT_DATA_ROOT = "../data/raw/Indonesian License Plate Dataset/"

input_images = os.path.join(INPUT_DATA_ROOT, "images")
input_labels = os.path.join(INPUT_DATA_ROOT, "labels")

# OUTPUT Paths (A new folder to store the augmented data, usually outside of 'raw')
OUTPUT_DATA_ROOT = "../data/processed/"

output_images = os.path.join(OUTPUT_DATA_ROOT, "images")
output_labels = os.path.join(OUTPUT_DATA_ROOT, "labels")

# --- END: Updated Paths ---

# Create output directories
os.makedirs(output_images, exist_ok=True)
os.makedirs(output_labels, exist_ok=True)

# Define augmentation pipeline
transform = A.Compose([
    A.Rotate(limit=5, p=0.5),
    A.Perspective(scale=(0.02, 0.05)),
    A.RandomBrightnessContrast(p=0.5),
    A.MotionBlur(blur_limit=5, p=0.3),
    A.RandomScale(scale_limit=0.2, p=0.5),
],
bbox_params=A.BboxParams(
    format='yolo', 
    label_fields=['class_labels'],
    # Add clip=True to ensure all coordinates are strictly within [0.0, 1.0]
    clip=True,
    min_area=1.0
))

# Loop through all images
for subdir in os.listdir(input_images):
    # Construct the full paths for the current subdirectory
    current_input_image_dir = os.path.join(input_images, subdir)
    current_input_label_dir = os.path.join(input_labels, subdir)

    # 🛑 FIX 2: Create the corresponding output subdirectories
    current_output_image_dir = os.path.join(output_images, subdir)
    current_output_label_dir = os.path.join(output_labels, subdir)

    os.makedirs(current_output_image_dir, exist_ok=True)
    os.makedirs(current_output_label_dir, exist_ok=True)
    
    print(f"Processing directory: {subdir}")

    # Loop through all images in the subdirectory
    # Using os.path.join is safer than f-string concatenation for paths
    for img_path in glob(os.path.join(current_input_image_dir, "*.jpg")):
        filename = os.path.basename(img_path)
        
        # 🛑 FIX 1: Correctly construct the label path using the subdirectory path
        label_path = os.path.join(current_input_label_dir, filename.replace(".jpg", ".txt"))

        # Safety Check: Ensure the label file exists before proceeding
        if not os.path.exists(label_path):
            print(f"Warning: Label file not found for {filename} in {subdir}. Skipping.")
            continue
            
        # Read image and labels (continue with your original logic)
        image = cv2.imread(img_path)
        if image is None:
            print(f"Error: Could not read image {img_path}. Skipping.")
            continue
            
        h, w, _ = image.shape # This is for context, not strictly needed by the augmentation code

        # ... (rest of the label reading, augmentation, and saving logic) ...

        with open(label_path, "r") as f:
            labels = f.readlines()

        bboxes = []
        class_labels = []
        for label in labels:
            try:
                cls, x, y, bw, bh = map(float, label.strip().split())
                bboxes.append([x, y, bw, bh])
                class_labels.append(int(cls))
            except Exception as e:
                 # Added error handling for malformed lines
                print(f"Error processing label line in {label_path}: {e}. Skipping image.")
                continue

        # Apply transformation
        transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)

        aug_img = transformed['image']
        aug_bboxes = transformed['bboxes']
        aug_labels = transformed['class_labels']

        # 🛑 FIX 2 (Final Save): Save augmented files into the correct OUTPUT subdirectory
        
        # Adding 'aug_' prefix is good practice to distinguish augmented files
        aug_filename = f"aug_{filename}" 

        # Save augmented image
        save_path_img = os.path.join(current_output_image_dir, aug_filename)
        cv2.imwrite(save_path_img, aug_img)

        # Save updated YOLO labels
        save_path_lbl = os.path.join(current_output_label_dir, aug_filename.replace(".jpg", ".txt"))
        with open(save_path_lbl, "w") as f:
            for cls, bbox in zip(aug_labels, aug_bboxes):
                f.write(f"{cls} {' '.join(map(str, bbox))}\n")

print("✅ Data augmentation complete!")


Processing directory: test
Processing directory: train
Processing directory: val
✅ Data augmentation complete!


In [2]:
from ultralytics import YOLO
import torch

print("YOLO version:", YOLO._version)
print("CUDA available:", torch.cuda.is_available())
print("Device being used:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


YOLO version: 1
CUDA available: True
Device being used: NVIDIA GeForce RTX 3060 Laptop GPU


In [10]:
DATA_YAML_PATH = '../data.yaml'

target_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load a pre-trained YOLOv8-nano model
model = YOLO('yolov8n.pt')

model.model.to(target_device) 

# 3. Verify the change (using the reliable check)
first_param_device = next(model.model.parameters()).device

print("✅ Model device (PyTorch check):", first_param_device)

print("Starting YOLOv8 training...")
results = model.train(
    data=DATA_YAML_PATH,     # Path to the data configuration file
    epochs=40,              # Number of epochs (adjust as needed)
    imgsz=640,               # Input image size (standard for YOLO)
    batch=6,                # Batch size (reduce if you run out of GPU memory)
    name='lp_detection_v2',  # Name for the results folder in 'runs/detect
    # Optional: use the 'freeze' argument if you want to freeze the backbone 
    amp=True,
    device=0,  
    lr0=1e-3
)

print("Training complete. Results saved in runs/detect/lp_detection_v2")

✅ Model device (PyTorch check): cuda:0
Starting YOLOv8 training...
New https://pypi.org/project/ultralytics/8.3.233 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.223 🚀 Python-3.10.19 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=6, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=../data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937,

In [11]:
evaluate = model.val()

Ultralytics 8.3.223 🚀 Python-3.10.19 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)


Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 2.2±0.3 ms, read: 141.0±11.5 MB/s, size: 1902.6 KB)
val: Scanning /mnt/d/Uni/Semester 5/Computer Vision/Plate-Recognition/data/processed/labels/val.cache... 100 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 100/100 88.6Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 7/7 0.2it/s 29.1s0.2s
                   all        100        179      0.956      0.979      0.991      0.715
Speed: 32.1ms preprocess, 82.9ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to /mnt/d/Uni/Semester 5/Computer Vision/Plate-Recognition/notebooks/runs/detect/val3


In [12]:
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv("runs/detect/lp_detection_v2/results.csv")

# Inspect columns and head to see what's actually in the CSV
print("columns:", df.columns.tolist())
print(df.head().T)

# Pick the mAP column robustly
map_cols = [c for c in df.columns if "mAP50" in c or "map50" in c.lower() or "mAP" in c]
if not map_cols:
    raise ValueError("No mAP column found in results.csv. Columns: " + ", ".join(df.columns))
col = map_cols[0]
print("Using metric column:", col)

# Convert to numeric (coerce errors -> NaN) and warn if any NaNs
y = pd.to_numeric(df[col], errors="coerce")
if y.isnull().any():
    print("Warning: some values in the metric column could not be converted to numeric and became NaN")

# Use epoch column if present else use dataframe index
x = df["epoch"] if "epoch" in df.columns else df.index

plt.figure(figsize=(6,4))
plt.plot(x, y, marker="o")
plt.xlabel("epoch" if "epoch" in df.columns else "index")
plt.ylabel(col)
plt.grid(True)
plt.tight_layout()
plt.show()

columns: ['epoch', 'time', 'train/box_loss', 'train/cls_loss', 'train/dfl_loss', 'metrics/precision(B)', 'metrics/recall(B)', 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'val/box_loss', 'val/cls_loss', 'val/dfl_loss', 'lr/pg0', 'lr/pg1', 'lr/pg2']
                              0          1          2          3          4
epoch                  1.000000   2.000000   3.000000   4.000000   5.000000
time                  20.588400  38.486100  56.903300  74.749800  94.643600
train/box_loss         1.556410   1.486290   1.503490   1.477920   1.428210
train/cls_loss         2.830230   1.750450   1.549060   1.302580   1.183260
train/dfl_loss         1.148850   1.111730   1.127920   1.111330   1.089600
metrics/precision(B)   0.797860   0.854530   0.868920   0.881150   0.928230
metrics/recall(B)      0.727690   0.810060   0.843580   0.882680   0.843580
metrics/mAP50(B)       0.822970   0.898340   0.910480   0.918690   0.918200
metrics/mAP50-95(B)    0.498420   0.536240   0.520880   0.569900   0.

<Figure size 600x400 with 1 Axes>

In [13]:
import os
import json
import cv2
from ultralytics import YOLO

YOLO_MODEL_PATH = "./runs/detect/lp_detection_v2/weights/best.pt"
INPUT_DIR = "../data/processed/images/"
OUTPUT_DIR = "../data/cropped/"

os.makedirs(OUTPUT_DIR, exist_ok=True)

model = YOLO(YOLO_MODEL_PATH)

mapping = {}

for class_name in os.listdir(INPUT_DIR):
    input_class_dir = os.path.join(INPUT_DIR, class_name)
    output_class_dir = os.path.join(OUTPUT_DIR, class_name)

    # Create output directory for this class
    os.makedirs(output_class_dir, exist_ok=True)

    for filename in os.listdir(input_class_dir):
        if not filename.lower().endswith(('.jpg', '.png', '.jpeg')):
            continue

        img_path = os.path.join(input_class_dir, filename)
        img = cv2.imread(img_path)
        if img is None:
            continue

        results = model.predict(img, verbose=False)
        boxes = results[0].boxes

        bboxes = boxes.xyxy.cpu().numpy()
        confs = boxes.conf.cpu().numpy()

        cropped_entries = []  # reset for each image

        for i, (x1, y1, x2, y2) in enumerate(bboxes[:, :4]):
            conf = confs[i]
            if conf < 0.5:
                continue

            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            crop = img[y1:y2, x1:x2]

            crop_name = f"{os.path.splitext(filename)[0]}_plate{i+1}.jpg"
            crop_path = os.path.join(output_class_dir, crop_name)

            cv2.imwrite(crop_path, crop)

            cropped_entries.append({
                "crop": crop_name,
                "confidence": float(conf),
                "bbox": [x1, y1, x2, y2]
            })

        if cropped_entries:
            mapping[filename] = cropped_entries

# Save mapping once at the end
mapping_file = os.path.join(OUTPUT_DIR, "detections.json")
with open(mapping_file, "w") as f:
    json.dump(mapping, f, indent=4)

print("Done!")


Done!


In [28]:
import cv2
import numpy as np
import math

def preprocess_for_ocr(img, target_h=32, target_w=128):
    """
    Preprocess cropped license plate for OCR:
    1. Grayscale
    2. Denoise (bilateral filter)
    3. Deskew (rotation estimation using moments)
    4. Resize + Normalize
    """

    # --- Step 1: Convert to Grayscale ---
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # --- Step 2: Denoise ---
    # Bilateral filter removes noise while keeping edges (better for characters)
    denoised = cv2.bilateralFilter(gray, d=7, sigmaColor=30, sigmaSpace=30)

    # --- Step 3: Deskew ---
    # Calculate image moments to estimate skew angle
    moments = cv2.moments(denoised)
    if abs(moments["mu02"]) < 1e-2:  
        # Avoid division by zero for clean horizontal text
        deskewed = denoised  
    else:
        skew = moments["mu11"] / moments["mu02"]
        angle = math.degrees(math.atan(skew))

        # Rotate opposite to deskew
        h, w = denoised.shape
        M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1)
        deskewed = cv2.warpAffine(denoised, M, (w, h), 
                                  flags=cv2.INTER_LINEAR, 
                                  borderMode=cv2.BORDER_REPLICATE)

    # --- Step 4: Resize for OCR ---
    # CRNN expects height=32, width flexible (often 100–160)
    resized = cv2.resize(deskewed, (target_w, target_h), interpolation=cv2.INTER_AREA)

    # --- Step 5: Normalize (0–1 float32) ---
    normalized = resized.astype("float32") / 255.0

    # Expand dims for OCR models expecting (1, H, W)
    normalized = np.expand_dims(normalized, axis=0)  # shape: (1, 32, 128)

    return np.squeeze(normalized)


<h1> Preprocess OCR dataset

#item semua coy


In [29]:
cropped_OCR_dir = "../data/raw/Indonesian License Plate Recognition Dataset/images/"
output_OCR_dir = "../data/trainOCR/images/"
os.makedirs(output_OCR_dir, exist_ok=True)

for dataset_category in os.listdir(cropped_OCR_dir):
    full_dataset_category = f"{cropped_OCR_dir}{dataset_category}/"
    output_category_dir = f"{output_OCR_dir}{dataset_category}/"
    os.makedirs(output_category_dir, exist_ok=True)
    for filename in os.listdir(full_dataset_category):
        full_cropped_image_path = f"{full_dataset_category}{filename}"
        full_output_image_path = f"{output_category_dir}{filename}"
        image = cv2.imread(full_cropped_image_path)
        processed_img = preprocess_for_ocr(img)
        # if processed_img is None:
        #     print("❌ ERROR: preprocess_for_ocr() returned None for:", full_cropped_image_path)
        #     continue

        # if not isinstance(processed_img, (list, tuple)) and processed_img.size == 0:
        #     print("❌ ERROR: Empty processed image for:", full_cropped_image_path)
        #     continue

        # print("Processed image shape:", processed_img.shape)

        # # Save preprocessed output
        # # output_path = os.path.join(output_category_dir, filename)
        cv2.imwrite(full_output_image_path, processed_img )

print("\n🎉 All license plates have been preprocessed for OCR!")


[ WARN:0@3204.333] global loadsave.cpp:1063 imwrite_ Unsupported depth image for selected encoder is fallbacked to CV_8U.


KeyboardInterrupt: 

<h1> Import dataset for OCR recognition

In [ ]:
import os
import cv2

# ------------------------------
# Paths
# ------------------------------
OCR_DATASET_DIR = "../data/raw/Indonesian License Plate Recognition Dataset/"
IMAGES_DIR = os.path.join(OCR_DATASET_DIR, "images/")
LABELS_DIR = os.path.join(OCR_DATASET_DIR, "labels/")
CLASSES_FILE = os.path.join(OCR_DATASET_DIR, "classes.names")


# ------------------------------
# Load class names
# ------------------------------
def load_class_names(path):
    with open(path, "r") as f:
        classes = [line.strip() for line in f.readlines() if line.strip()]
    return classes


# ------------------------------
# Convert YOLO (normalised) → Pixel BBox
# ------------------------------
def yolo_to_xyxy(yolo_bbox, img_w, img_h):
    """
    yolo_bbox = (cx, cy, w, h) normalized
    return: x1, y1, x2, y2 in pixel coordinates
    """
    cx, cy, w, h = yolo_bbox

    # Convert to pixel values
    cx *= img_w
    cy *= img_h
    w *= img_w
    h *= img_h

    x1 = int(cx - w / 2)
    y1 = int(cy - h / 2)
    x2 = int(cx + w / 2)
    y2 = int(cy + h / 2)

    return max(0, x1), max(0, y1), min(img_w, x2), min(img_h, y2)


# ------------------------------
# Load dataset
# ------------------------------
def load_ocr_dataset():
    dataset = []

    class_names = load_class_names(CLASSES_FILE)
    for dataset_category in os.lisdir(IMAGES_DIR):
        full_dataset_category = f"{IMAGES_DIR}{dataset_category}/"
        full_label_dir = f"{IMAGES_DIR}{dataset_category}/"
        for filename in os.listdir(full_dataset_category):
            if not filename.lower().endswith((".jpg", ".png", ".jpeg")):
                continue

            img_path = os.path.join(full_dataset_category, filename)
            print(img_path)
            lbl_path = os.path.join(full_label_dir, filename.replace(".jpg", ".txt").replace(".png", ".txt"))

            if not os.path.exists(lbl_path):
                print(f"⚠ No label file found for: {filename}")
                continue

            # Load image
            img = cv2.imread(img_path)
            img_h, img_w = img.shape[:2]

            # Load YOLO labels
            boxes = []
            with open(lbl_path, "r") as f:
                lines = f.readlines()

            for line in lines:
                parts = line.strip().split()
                if len(parts) < 5:
                    print(f"⚠ Skipping invalid label line in {lbl_path}: {line}")
                    continue

                cls_id = int(parts[0])
                cx, cy, w, h = map(float, parts[1:5])

                x1, y1, x2, y2 = yolo_to_xyxy((cx, cy, w, h), img_w, img_h)

                boxes.append({
                    "class_id": cls_id,
                    "class_name": class_names[cls_id] if cls_id < len(class_names) else "UNKNOWN",
                    "bbox_normalized": [cx, cy, w, h],
                    "bbox_xyxy": [x1, y1, x2, y2]
                })
                print(boxes)

            dataset.append({
                "image_path": img_path,
                "image": img,
                "boxes": boxes
            })

        print(f"📦 Loaded {len(dataset)} images with labels.")
        return dataset


# ------------------------------
# Example Usage
# ------------------------------
if __name__ == "__main__":
    data = load_ocr_dataset()
    print(data[0])  # Show first dataset entry


📦 Loaded 0 images with labels.


IndexError: list index out of range

In [10]:
class_names = load_class_names(CLASSES_FILE)
print(class_names)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
